In [2]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np
import os

import re
import io

import psycopg2
from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int64, AsIs)
#from config import config
from configparser import ConfigParser

In [3]:
# url for all games played on date - https://www.basketball-reference.com/boxscores/?month=3&day=15&year=2024
# url for box score of specific game - https://www.basketball-reference.com/boxscores/202403150CHO.html

# Which info do we need?
# 1. How many games played
# 2. Final Score for each
# 3. Winner / Loser for each


In [4]:
games_on_date_test_url = 'https://www.basketball-reference.com/boxscores/?month=3&day=15&year=2024'

games_on_date_test_page = requests.get(games_on_date_test_url,params={
      'render_js': True})    

game_on_date_test_soup = BeautifulSoup(games_on_date_test_page.text, 'lxml')

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [4]:
game_on_date_test_soup.find_all('tr')[0].find('a', href=re.compile('/boxscores/'))

<a href="/boxscores/202403150CHO.html">F<span class="no_mobile">inal</span></a>

In [5]:
game_on_date_test_soup.find_all('tr', {'class':['winner', 'loser']})[0].find('a', href=re.compile('/boxscores/'))

<a href="/boxscores/202403150CHO.html">F<span class="no_mobile">inal</span></a>

In [6]:
for i in range(len(game_on_date_test_soup.find_all('tr', {'class':['winner', 'loser']}))):
    try:
        print(game_on_date_test_soup.find_all('tr', {'class':['winner', 'loser']})[i].find('a', href=re.compile('/boxscores/')).attrs['href'].split('/')[2].split('.')[0][-3:])
    except:
        pass

CHO
DET
NOP
SAS
TOR
UTA


In [7]:
print(game_on_date_test_soup)

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/deploy/www" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://cdn.ssref.net/req/202501061" rel="dns-prefetch"/>
<script>
/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = window.dataLayer ||[];
      function gtag(){dataLayer.push(arguments);}
      gtag('consent','default',{
        'ad_storage':'denied',
        'analytics_storage':'denied',
        'ad_user_data':'denied',
        'ad_personalization':'denied',
        'personalization_storage':'denied',
        'functionality_storage':'granted',
        'security_storage':'granted',
        'wait_for_update': 500
      });
      gtag("set", "ads_data_redaction", true);
</script>
<script src="https://cmp.osano.com/16CGnCU8UtNhM14sg/12669873-8c

In [8]:
game_on_date_test_soup.find_all('a', href=re.compile('/boxscores/'))

[<a href="/boxscores/">Scores</a>,
 <a href="/boxscores/" itemprop="item"><span itemprop="name">Box Scores</span></a>,
 <a class="button2 prev" href="/boxscores/?month=3&amp;day=14&amp;year=2024">Mar 14, 2024</a>,
 <a class="button2 next" href="/boxscores/?month=3&amp;day=16&amp;year=2024">Mar 16, 2024</a>,
 <a href="/boxscores/202403150CHO.html">F<span class="no_mobile">inal</span></a>,
 <a href="/boxscores/202403150CHO.html">Box Score</a>,
 <a href="/boxscores/pbp/202403150CHO.html">Play-By-Play</a>,
 <a href="/boxscores/shot-chart/202403150CHO.html">Shot Chart</a>,
 <a href="/boxscores/202403150DET.html">F<span class="no_mobile">inal</span></a>,
 <a href="/boxscores/202403150DET.html">Box Score</a>,
 <a href="/boxscores/pbp/202403150DET.html">Play-By-Play</a>,
 <a href="/boxscores/shot-chart/202403150DET.html">Shot Chart</a>,
 <a href="/boxscores/202403150NOP.html">F<span class="no_mobile">inal</span></a>,
 <a href="/boxscores/202403150NOP.html">Box Score</a>,
 <a href="/boxscores/p

In [9]:
# winning team score
print(int(game_on_date_test_soup.find('tbody').find_all('td')[1].contents[0]))

# winning team name
print(game_on_date_test_soup.find('tbody').find_all('td')[0].find('a').attrs['href'].split('/')[2])

# losing team score
print(int(game_on_date_test_soup.find('tbody').find_all('td')[4].contents[0]))

# losing team name
print(game_on_date_test_soup.find('tbody').find_all('td')[3].find('a').attrs['href'].split('/')[2])


107
PHO
96
CHO


In [10]:
print(int(game_on_date_test_soup.find_all('tbody')[1].find_all('td')[1].contents[0]))

28


In [11]:
print(game_on_date_test_soup.find_all('tbody'))

[<tbody>
<tr class="winner">
<td><a href="/teams/PHO/2024.html">Phoenix</a></td>
<td class="right">107</td>
<td class="right gamelink">
<a href="/boxscores/202403150CHO.html">F<span class="no_mobile">inal</span></a>
</td>
</tr>
<tr class="loser">
<td><a href="/teams/CHO/2024.html">Charlotte</a></td>
<td class="right">96</td>
<td class="right"> 
			</td>
</tr>
</tbody>, <tbody>
<tr>
<td><a href="/teams/PHO/2024.html">Phoenix</a> </td>
<td class="center">28</td><td class="center">26</td><td class="center">38</td><td class="center">15</td>
</tr>
<tr>
<td><a href="/teams/CHO/2024.html">Charlotte</a> </td>
<td class="center">26</td><td class="center">19</td><td class="center">22</td><td class="center">29</td>
</tr>
</tbody>, <tbody>
<tr>
<td><strong>PTS</strong></td>
<td>2 tied</td>
<td class="right">21</td>
</tr>
<tr>
<td><strong>TRB</strong></td>
<td><a href="/players/n/nurkiju01.html">J. Nurkić</a>-PHO</td>
<td class="right">21</td>
</tr>
</tbody>, <tbody>
<tr class="winner">
<td><a href

In [12]:
game_on_date_test_soup.find_all('tr', {"class": "winner"})[0]

<tr class="winner">
<td><a href="/teams/PHO/2024.html">Phoenix</a></td>
<td class="right">107</td>
<td class="right gamelink">
<a href="/boxscores/202403150CHO.html">F<span class="no_mobile">inal</span></a>
</td>
</tr>

In [13]:
game_on_date_test_soup.find_all('tr')[2]

<tr>
<th></th><!-- the team labels -->
<th>1</th><th>2</th><th>3</th><th>4</th>
</tr>

In [14]:
game_on_date_test_soup.find_all('tr', {"class": "winner"})[1].find_all('a')[1].attrs['href'].split('/')[2].split('.')[0][-3:]

'DET'

In [15]:
game_on_date_test_soup.find_all('tr', {"class": "winner"})[0].find_all('td')[0].find('a').attrs['href'].split('/')[2]

'PHO'

In [16]:
#print(game_on_date_test_soup.find('tbody').find_all('td')[0].find('a').attrs['href'].split('/')[2])
print(int(game_on_date_test_soup.find_all('tr', {"class": "winner"})[3].find_all('td')[1].contents[0]))

117


In [17]:
date = '20230315'

year = date[:4]
month = date[4:6]
day = date[-2:]

print(year + " " + month + " " + day)

2023 03 15


In [18]:
# Probably best to create individual series for each datapoint, ie loser score, then create the df at the end

def get_game_results(date):
    
    # need to incorporate game date into function

    #yyyymmdd
    #date = '20230315'

    year = date[:4]
    month = date[4:6]
    day = date[-2:]
    
    games_on_date_test_url = 'https://www.basketball-reference.com/boxscores/?month='  + str(int(month)) + '&day=' + str(int(day)) + '&year=' + str(int(year))

    games_on_date_test_page = requests.get(games_on_date_test_url,params={
      'render_js': True})    

    game_on_date_test_soup = BeautifulSoup(games_on_date_test_page.text, 'lxml')

    #game_results_df = pd.DataFrame()

    winning_teams = []
    losing_teams = []
    winning_scores = []
    losing_scores = []
    home_teams = []

    for i in range(len(game_on_date_test_soup.find_all('tr', {"class": "winner"}))):
        
      winning_teams.append(game_on_date_test_soup.find_all('tr', {"class": "winner"})[i].find_all('td')[0].find('a').attrs['href'].split('/')[2])
      losing_teams.append(game_on_date_test_soup.find_all('tr', {"class": "loser"})[i].find_all('td')[0].find('a').attrs['href'].split('/')[2])

      winning_scores.append(int(game_on_date_test_soup.find_all('tr', {"class": "winner"})[i].find_all('td')[1].contents[0]))
      losing_scores.append(int(game_on_date_test_soup.find_all('tr', {"class": "loser"})[i].find_all('td')[1].contents[0]))

      #home_teams.append(game_on_date_test_soup.find_all('tr', {"class": "winner"})[i].find_all('a')[1].attrs['href'].split('/')[2].split('.')[0][-3:])

    for i in range(len(game_on_date_test_soup.find_all('tr', {'class':['winner', 'loser']}))):
      try:
        home_teams.append(game_on_date_test_soup.find_all('tr', {'class':['winner', 'loser']})[i].find('a', href=re.compile('/boxscores/')).attrs['href'].split('/')[2].split('.')[0][-3:])
      except:
        pass

    temp_df = pd.DataFrame({'home': home_teams, 'winning_team': winning_teams, 'losing_team': losing_teams, 'winning_score': winning_scores, 'losing_score': losing_scores})
    
    temp_df['visitor'] = temp_df.apply(lambda x: x.winning_team if x.winning_team != x.home else x.losing_team, axis = 1)

    temp_df['visitor_score'] = temp_df.apply(lambda x: x.winning_score if x.winning_team != x.home else x.losing_score, axis = 1)

    temp_df['home_score'] = temp_df.apply(lambda x: x.winning_score if x.winning_team == x.home else x.losing_score, axis = 1)

    temp_df['dates'] = date

    temp_df['id'] = temp_df['visitor'] + temp_df['home'] + temp_df['dates']

    final_df = temp_df[['id', 'visitor', 'home', 'dates', 'visitor_score', 'home_score']]

    #temp_df['visiting_team'] = temp_df.apply(lambda x: x.winning_team if x.winning_team != x.home_team else x.losing_team)#[x for x in temp_df['winning_team']]
    return(final_df)
    #print(winning_teams)
    #print(losing_teams)
    #print(winning_scores)
    #print(losing_scores)
    #print(home_teams)

    # id = visitor initial + home initial + YYYYMMDD
    # visitor
    # home
    # date
    # visitor score
    # home score


In [19]:
# Probably best to create individual series for each datapoint, ie loser score, then create the df at the end

def get_game_results_w_api(date):
    
    # need to incorporate game date into function

    #yyyymmdd
    #date = '20230315'

    year = date[:4]
    month = date[4:6]
    day = date[-2:]
    
    games_on_date_test_url = 'https://www.basketball-reference.com/boxscores/?month='  + str(int(month)) + '&day=' + str(int(day)) + '&year=' + str(int(year))

    payload = { 'api_key': '1e39e30a6e96d4b3c90d44ae0e3222e2', 'url': games_on_date_test_url, 'country_code': 'us', 'device_type': 'desktop', 'max_cost': '1000', 'session_number': '0', 'render_js': True}
    
    games_on_date_test_page = requests.get('https://api.scraperapi.com/', params=payload)

    #games_on_date_test_page = requests.get(games_on_date_test_url,params={
    #  'render_js': True})   

    game_on_date_test_soup = BeautifulSoup(games_on_date_test_page.text, 'lxml')

    #game_results_df = pd.DataFrame()

    winning_teams = []
    losing_teams = []
    winning_scores = []
    losing_scores = []
    home_teams = []

    for i in range(len(game_on_date_test_soup.find_all('tr', {"class": "winner"}))):
        
      winning_teams.append(game_on_date_test_soup.find_all('tr', {"class": "winner"})[i].find_all('td')[0].find('a').attrs['href'].split('/')[2])
      losing_teams.append(game_on_date_test_soup.find_all('tr', {"class": "loser"})[i].find_all('td')[0].find('a').attrs['href'].split('/')[2])

      winning_scores.append(int(game_on_date_test_soup.find_all('tr', {"class": "winner"})[i].find_all('td')[1].contents[0]))
      losing_scores.append(int(game_on_date_test_soup.find_all('tr', {"class": "loser"})[i].find_all('td')[1].contents[0]))

      #home_teams.append(game_on_date_test_soup.find_all('tr', {"class": "winner"})[i].find_all('a')[1].attrs['href'].split('/')[2].split('.')[0][-3:])

    for i in range(len(game_on_date_test_soup.find_all('tr', {'class':['winner', 'loser']}))):
      try:
        home_teams.append(game_on_date_test_soup.find_all('tr', {'class':['winner', 'loser']})[i].find('a', href=re.compile('/boxscores/')).attrs['href'].split('/')[2].split('.')[0][-3:])
      except:
        pass

    temp_df = pd.DataFrame({'home': home_teams, 'winning_team': winning_teams, 'losing_team': losing_teams, 'winning_score': winning_scores, 'losing_score': losing_scores})
    
    temp_df['visitor'] = temp_df.apply(lambda x: x.winning_team if x.winning_team != x.home else x.losing_team, axis = 1)

    temp_df['visitor_score'] = temp_df.apply(lambda x: x.winning_score if x.winning_team != x.home else x.losing_score, axis = 1)

    temp_df['home_score'] = temp_df.apply(lambda x: x.winning_score if x.winning_team == x.home else x.losing_score, axis = 1)

    temp_df['dates'] = date

    temp_df['id'] = temp_df['visitor'] + temp_df['home'] + temp_df['dates']

    final_df = temp_df[['id', 'visitor', 'home', 'dates', 'visitor_score', 'home_score']]

    #temp_df['visiting_team'] = temp_df.apply(lambda x: x.winning_team if x.winning_team != x.home_team else x.losing_team)#[x for x in temp_df['winning_team']]
    return(final_df)
    #print(winning_teams)
    #print(losing_teams)
    #print(winning_scores)
    #print(losing_scores)
    #print(home_teams)

    # id = visitor initial + home initial + YYYYMMDD
    # visitor
    # home
    # date
    # visitor score
    # home score


In [20]:
get_game_results_w_api('20240315')

,id,visitor,home,dates,visitor_score,home_score
0,PHOCHO20240315,PHO,CHO,20240315,107,96
1,MIADET20240315,MIA,DET,20240315,108,95
2,LACNOP20240315,LAC,NOP,20240315,104,112
3,DENSAS20240315,DEN,SAS,20240315,117,106
4,ORLTOR20240315,ORL,TOR,20240315,113,103
5,ATLUTA20240315,ATL,UTA,20240315,122,124


In [21]:
get_game_results('20240315')

,id,visitor,home,dates,visitor_score,home_score
0,PHOCHO20240315,PHO,CHO,20240315,107,96
1,MIADET20240315,MIA,DET,20240315,108,95
2,LACNOP20240315,LAC,NOP,20240315,104,112
3,DENSAS20240315,DEN,SAS,20240315,117,106
4,ORLTOR20240315,ORL,TOR,20240315,113,103
5,ATLUTA20240315,ATL,UTA,20240315,122,124


In [6]:
def config(filename = 'database.ini', section = 'postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    db = {}

    if parser.has_section(section):
        params = parser.items(section)

        for param in params:
            db[param[0]] = param[1]

    else:
        raise Exception('Section{0} is not found in the {1} file'.format(section, filename))
    
    return(db)

config()

Exception: Sectionpostgresql is not found in the database.ini file

In [89]:
df_test = get_game_results('20240315')

for index in df_test.index:
    #print(tuple(df_test.iloc[row].values[0]))
    print(tuple(df_test.iloc[index]))

('PHOCHO20240315', 'PHO', 'CHO', '20240315', 107, 96)
('MIADET20240315', 'MIA', 'DET', '20240315', 108, 95)
('LACNOP20240315', 'LAC', 'NOP', '20240315', 104, 112)
('DENSAS20240315', 'DEN', 'SAS', '20240315', 117, 106)
('ORLTOR20240315', 'ORL', 'TOR', '20240315', 113, 103)
('ATLUTA20240315', 'ATL', 'UTA', '20240315', 122, 124)


In [5]:


def create_sql_tables():
    connection = None
    try:
        params = config()
        #print(params)
        print('connecting to postgresql database...')
        connection = psycopg2.connect(**params)
       
        # create cursor
        cursor = connection.cursor()
        
        # add all sql create table commands here
        #cursor.execute(open('create_game_results_table.sql', 'r').read())
        #cursor.execute(open('create_yearly_team_per_game_stats_table.sql', 'r').read())
        #cursor.execute(open('create_yearly_opponent_per_game_stats_table.sql', 'r').read())

        try:
            
            # add all sql create table commands here
            #cursor.execute(open('create_game_results_table.sql', 'r').read())
            #cursor.execute(open('create_yearly_team_per_game_stats_table.sql', 'r').read())
            #cursor.execute(open('create_yearly_opponent_per_game_stats_table.sql', 'r').read())


            connection.commit()
            print("connection successful / task complete")
            cursor.close()
            #os.remove(tmp_df)
            
        except (Exception, psycopg2.DatabaseError) as error:
            #os.remove(tmp_df)
            print("Error: %s" % error)
            #connection.rollback()
            cursor.close()
            return 1
    
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if connection is not None:
            connection.close()
            print('database connection terminated')

if __name__ == '__main__':
    create_sql_tables()


Sectionpostgresql is not found in the database.ini file


In [45]:


def get_dates_already_in_db():
    connection = None
    try:
        params = config()
        #print(params)
        print('connecting to postgresql database...')
        connection = psycopg2.connect(**params)
       
        # create cursor
        cursor = connection.cursor()

        try:
            
            cursor.execute("""SELECT DISTINCT(dates) FROM game_results;""")

            game_dates_already_in_db = cursor.fetchall()

            connection.commit()
            print("connection successful / task complete")
            cursor.close()
            return game_dates_already_in_db
            #os.remove(tmp_df)
            
        except (Exception, psycopg2.DatabaseError) as error:
            #os.remove(tmp_df)
            print("Error: %s" % error)
            #connection.rollback()
            cursor.close()
            return 1
    
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if connection is not None:
            connection.close()
            print('database connection terminated')

if __name__ == '__main__':
    get_dates_already_in_db()


connecting to postgresql database...
connection successful / task complete
database connection terminated


In [46]:
existing_game_dates_raw = get_dates_already_in_db()

connecting to postgresql database...
connection successful / task complete
database connection terminated


In [47]:
existing_game_dates_clean = [x[0] for x in existing_game_dates_raw]
existing_game_dates_clean.sort(reverse=True)

In [48]:
print(existing_game_dates_clean)

[20250107, 20250106, 20250105, 20250104, 20250103, 20250102, 20250101, 20241231, 20241230, 20241229, 20241228, 20241227, 20241226, 20241225, 20241223, 20241222, 20241221, 20241220, 20241219, 20241217, 20241216, 20241215, 20241214, 20241213, 20241212, 20241211, 20241210, 20241209, 20241208, 20241207, 20241206, 20241205, 20241204, 20241203, 20241202, 20241201, 20241130, 20241129, 20241127, 20241126, 20241125, 20241124, 20241123, 20241122, 20241121, 20241120, 20241119, 20241118, 20241117, 20241116, 20241115, 20241114, 20241113, 20241112, 20241111, 20241110, 20241109, 20241108, 20241107, 20241106, 20241104, 20241103, 20241102, 20241101, 20241031, 20241030, 20241029, 20241028, 20240614, 20240612, 20240609, 20240606, 20240530, 20240528, 20240527, 20240526, 20240525, 20240524, 20240523, 20240522, 20240521, 20240519, 20240518, 20240517, 20240516, 20240515, 20240514, 20240513, 20240512, 20240511, 20240510, 20240509, 20240508, 20240507, 20240506, 20240505, 20240504, 20240503, 20240502, 20240501,

In [49]:
regular_season_dates = {
  2000: [19991031, 20000416],
  2001: [20001031, 20010418],
  2002: [20011030, 20020417],
  2003: [20021029, 20030416],
  2004: [20031028, 20040414],
  2005: [20041102, 20050420],
  2006: [20051101, 20060419],
  2007: [20061031, 20070415],
  2008: [20071030, 20080416],
  2009: [20081028, 20090415],
  2010: [20091027, 20100414],
  2011: [20101026, 20110413],
  2012: [20111225, 20120426],
  2013: [20121030, 20130417],
  2014: [20131029, 20140416],
  2015: [20141028, 20150415],
  2016: [20151027, 20160413],
  2017: [20161025, 20170412],
  2018: [20171017, 20180411],
  2019: [20181016, 20190410],
  2020: [20191022, 20200311],
  2021: [20201222, 20210716],
  2022: [20211019, 20220410],
  2023: [20221018, 20230409],
  2024: [20231024, 20230414],
  2025: [20241028, 20250413]
}

playoff_dates = {
  2000: [20000421, 20000619],
  2001: [20010420, 20010615],
  2002: [20020419, 20020616],
  2003: [20030418, 20030615],
  2004: [20040416, 20040615],
  2005: [20050423, 20050623],
  2006: [20060421, 20060620],
  2007: [20070420, 20070614],
  2008: [20080418, 20080617],
  2009: [20090418, 20090614],
  2010: [20100417, 20100617],
  2011: [20110416, 20110612],
  2012: [20120428, 20120621],
  2013: [20130420, 20130620],
  2014: [20140419, 20140615],
  2015: [20150418, 20150616],
  2016: [20160416, 20160619],
  2017: [20170415, 20170612],
  2018: [20180414, 20180608],
  2019: [20190413, 20190613],
  2020: [20200817, 20201011],
  2021: [20210522, 20210720],
  2022: [20220416, 20220616],
  2023: [20230415, 20230618],
  2024: [20240420, 20240616],
  2025: [20250418, 20250615]
}

In [50]:


def add_game_data_to_db(date):
    connection = None
    try:
        params = config()
        #print(params)
        print('connecting to postgresql database...')
        connection = psycopg2.connect(**params)
       
        # create cursor
        cursor = connection.cursor()
        
        # example
        #print('postgresql database version: ')
        #crsr.execute('SELECT version()')
        #db_version = crsr.fetchone()
        #print(db_version)

        try:
            #cursor.copy_from(f, 'game_results', sep=",")
            #cursor.copy_from(f_21_22, 'game_results', sep=",")
            #cursor.copy_from(f_19, 'game_results', sep=",")
            #cursor.copy_from(f_17_18, 'game_results', sep=",")
            #cursor.copy_from(f_15_16, 'game_results', sep=",")
            #cursor.copy_from(f_13_14, 'game_results', sep=",")

            #cursor.copy_from(f_team_per_game_stats, 'yearly_team_per_game_stats', sep=",")
            #cursor.copy_from(f_opp_per_game_stats, 'yearly_opponent_per_game_stats', sep=",")
            
            #cursor.copy_from(final_df_csv, 'game_results')

            #cursor.execute("""INSERT INTO game_results (id, visitor, home, dates, visitor_score, home_score) VALUES (%s, %s, %s, %s, %s, %s); """, tuple(df_test.iloc[0]))

            game_data_on_date = get_game_results_w_api(str(date))
            for index in game_data_on_date.index:
                cursor.execute("""INSERT INTO game_results (id, visitor, home, dates, visitor_score, home_score) VALUES (%s, %s, %s, %s, %s, %s); """, tuple(game_data_on_date.iloc[index]))

            connection.commit()
            #print("connection successful / task complete")
            cursor.close()
            #os.remove(tmp_df)
            
        except (Exception, psycopg2.DatabaseError) as error:
            #os.remove(tmp_df)
            print("Error: %s" % error)
            #connection.rollback()
            cursor.close()
            return 1
    
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if connection is not None:
            connection.close()
           #print('database connection terminated')

#if __name__ == '__main__':
#    get_game_data('20240315')


In [51]:
#add_game_data_to_db('20240315')

In [60]:
from datetime import date, timedelta

valid_game_days = []

start_date = date.today() 
end_date = date(1999, 10, 31)    # perhaps date.today()

delta = start_date - end_date   # returns timedelta

for i in range(delta.days + 1):
    day = str(start_date - timedelta(days=i))
    day = int(day.replace("-", ""))
    month = (day // 100) % 100
    year = day // 10000
    if month > 8 and day >= regular_season_dates[year + 1][0]:
        valid_game_days.append(day)
    elif day <= playoff_dates[year][1]:
        valid_game_days.append(day)

In [61]:
valid_game_days

[20250108,
 20250107,
 20250106,
 20250105,
 20250104,
 20250103,
 20250102,
 20250101,
 20241231,
 20241230,
 20241229,
 20241228,
 20241227,
 20241226,
 20241225,
 20241224,
 20241223,
 20241222,
 20241221,
 20241220,
 20241219,
 20241218,
 20241217,
 20241216,
 20241215,
 20241214,
 20241213,
 20241212,
 20241211,
 20241210,
 20241209,
 20241208,
 20241207,
 20241206,
 20241205,
 20241204,
 20241203,
 20241202,
 20241201,
 20241130,
 20241129,
 20241128,
 20241127,
 20241126,
 20241125,
 20241124,
 20241123,
 20241122,
 20241121,
 20241120,
 20241119,
 20241118,
 20241117,
 20241116,
 20241115,
 20241114,
 20241113,
 20241112,
 20241111,
 20241110,
 20241109,
 20241108,
 20241107,
 20241106,
 20241105,
 20241104,
 20241103,
 20241102,
 20241101,
 20241031,
 20241030,
 20241029,
 20241028,
 20240616,
 20240615,
 20240614,
 20240613,
 20240612,
 20240611,
 20240610,
 20240609,
 20240608,
 20240607,
 20240606,
 20240605,
 20240604,
 20240603,
 20240602,
 20240601,
 20240531,
 20240530,

In [65]:
days_already_tried_adding = []

In [66]:
missing_game_days = list(set(valid_game_days) - set(existing_game_dates_clean) - set(days_already_tried_adding))

In [67]:
missing_game_days.sort(reverse=True)

In [68]:
missing_game_days

[20250108,
 20241224,
 20241218,
 20241128,
 20241105,
 20240616,
 20240615,
 20240613,
 20240611,
 20240610,
 20240608,
 20240607,
 20240605,
 20240604,
 20240603,
 20240602,
 20240601,
 20240531,
 20240529,
 20240520,
 20240418,
 20240415,
 20240413,
 20240408,
 20240221,
 20240220,
 20240219,
 20240218,
 20240217,
 20240216,
 20231224,
 20231210,
 20231203,
 20231123,
 20231107,
 20230618,
 20230617,
 20230616,
 20230615,
 20230614,
 20230613,
 20230611,
 20230610,
 20230608,
 20230606,
 20230605,
 20230603,
 20230602,
 20230531,
 20230530,
 20230528,
 20230526,
 20230524,
 20230515,
 20230513,
 20230413,
 20230410,
 20230403,
 20230222,
 20230221,
 20230220,
 20230219,
 20230218,
 20230217,
 20221224,
 20221124,
 20221108,
 20220615,
 20220614,
 20220612,
 20220611,
 20220609,
 20220607,
 20220606,
 20220604,
 20220603,
 20220601,
 20220531,
 20220530,
 20220528,
 20220516,
 20220514,
 20220505,
 20220430,
 20220414,
 20220411,
 20220404,
 20220223,
 20220222,
 20220221,
 20220220,

In [71]:
for day in missing_game_days:
    if day < 20040301:
        try:
            add_game_data_to_db(day)
            days_already_tried_adding.append(day)
            print(" successfully added game data for " + str(day) + " to the database")
        except:
            print(" failed to add game data for " + str(day) + " to the database")

connecting to postgresql database...
Error: duplicate key value violates unique constraint "game_results_pkey"
DETAIL:  Key (id)=(NYKDEN20040229) already exists.

 successfully added game data for 20040229 to the database
connecting to postgresql database...
 successfully added game data for 20040228 to the database
connecting to postgresql database...
Error: duplicate key value violates unique constraint "game_results_pkey"
DETAIL:  Key (id)=(TORBOS20040227) already exists.

 successfully added game data for 20040227 to the database
connecting to postgresql database...
Error: duplicate key value violates unique constraint "game_results_pkey"
DETAIL:  Key (id)=(SASDAL20040226) already exists.

 successfully added game data for 20040226 to the database
connecting to postgresql database...
Error: duplicate key value violates unique constraint "game_results_pkey"
DETAIL:  Key (id)=(MILBOS20040225) already exists.

 successfully added game data for 20040225 to the database
connecting to po

/var/folders/kc/tfm_mmns5096hfr18knr3rp40000gp/T/ipykernel_19474/4121140257.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  game_on_date_test_soup = BeautifulSoup(games_on_date_test_page.text, 'lxml')


 successfully added game data for 20000117 to the database
connecting to postgresql database...


/var/folders/kc/tfm_mmns5096hfr18knr3rp40000gp/T/ipykernel_19474/4121140257.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  game_on_date_test_soup = BeautifulSoup(games_on_date_test_page.text, 'lxml')


 successfully added game data for 20000116 to the database
connecting to postgresql database...


...

/var/folders/kc/tfm_mmns5096hfr18knr3rp40000gp/T/ipykernel_19474/4121140257.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  game_on_date_test_soup = BeautifulSoup(games_on_date_test_page.text, 'lxml')


 successfully added game data for 19991102 to the database
connecting to postgresql database...


/var/folders/kc/tfm_mmns5096hfr18knr3rp40000gp/T/ipykernel_19474/4121140257.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  game_on_date_test_soup = BeautifulSoup(games_on_date_test_page.text, 'lxml')


 successfully added game data for 19991101 to the database
connecting to postgresql database...
 successfully added game data for 19991031 to the database


/var/folders/kc/tfm_mmns5096hfr18knr3rp40000gp/T/ipykernel_19474/4121140257.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  game_on_date_test_soup = BeautifulSoup(games_on_date_test_page.text, 'lxml')


In [1]:
get_game_results('20240923')

NameError: name 'get_game_results' is not defined

,id,visitor,home,dates,visitor_score,home_score


In [ ]:
#todo

#create a list of all valid days in format yyyy/mm/dd

#create a table of playoff start and end dates (do this for the regular season too if needed)

#look into parsing additional (more granular) game data

In [33]:
def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = io.StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()

In [34]:
copy_from_stringio(connect(), final_df, 'game_results')

connecting to postgresql database...
copy_from_file() done
database connection terminated


AttributeError: 'NoneType' object has no attribute 'cursor'